# 유저 후보 응답 생성
#### 현재 상황은 아래와 같습니다.
1. 현재 대화의 진행방식은 유저가 말하고, 에이전트가 말하는 방식입니다. 그렇기 때문에 유저가 먼저 말을 해야하는데, 처음 우리 앱을 들어온 유저는 우리가 어떤 것을 하는지 전혀 모르기 때문에 이에 대해 가이드를 주어야합니다.
2. 추천에 있어서는 에이전트가 유저를 분석하기 위해 여러 와인 선정 기준에 대해 질문을 합니다.(예시: 와인의 종류는 어떤것을 좋아하시나요?) 이런 질문에 대해서 후보 답변을 주는 것은 유저의 선택에 도움이 될 것 입니다.(예시: 1. 레드와인, 2. 화이트와인, 3. 스파클링)

#### 생각할 수 있는 해결방식은 아래와 같습니다.
1. 적절한 유저 프롬프트를 작성하고, Langchain을 활용하여 유저의 예시 답변을 생성한다.
2. 미리 example들을 작성해두고, 이를 프롬프트에 추가하는 few shot방식을 통해 성능이 올라갈 수 있다.

#### 참고 사항
아래는 위의 작업을 수행하기 위한 좋은 자료들입니다. 위의 문제해결 전에 아래 자료들을 먼저 학습하는 것을 매우 강력하게 추천합니다.
- [프롬프트 엔지니어링 강의 2시간](https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/)
- [프롬프트 엔지니어링 가이드 documentation](https://www.promptingguide.ai/techniques)

-------------------------------------------------------------------------

### API 키 불러오기

In [1]:
import os
import configparser
import time
import copy

In [2]:
config = configparser.ConfigParser()
config.read('../secrets.ini')

['../secrets.ini']

In [3]:
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
serper_api_key = config['SERPER']['SERPER_API_KEY']
serp_api_key = config['SERPAPI']['SERPAPI_API_KEY']
os.environ.update({'OPENAI_API_KEY': openai_api_key})
os.environ.update({'SERPER_API_KEY': serper_api_key})
os.environ.update({'SERPAPI_API_KEY': serp_api_key})

### Get User Prompt

In [13]:
from typing import List, Union
import re
import json

import pandas as pd
from langchain import SerpAPIWrapper, LLMChain
from langchain.agents import Tool, AgentType, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.document_loaders import DataFrameLoader, SeleniumURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.prompts import PromptTemplate, StringPromptTemplate
from langchain.prompts import load_prompt, BaseChatPromptTemplate
from langchain.prompts.loading import load_prompt_from_config
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.schema import AgentAction, AgentFinish, HumanMessage
from langchain.vectorstores import DocArrayInMemorySearch, Chroma

유저 프롬프트 템플릿은 few_shot prompt를 사용하기 위해 [prefix template text file](../model/templates/user_response_prompt_prefix_template.txt)와 [suffix template text file](../model/templates/user_response_prompt_suffix_template.txt)로 나누어 작성합니다.

유저 프롬프트에 넣을 예시는 [examples text file](../model/templates/examples/user_response_prompt_examples.json)에 작성합니다.

그리고 [user prompt json file](../model/templates/user_response_prompt.json)을 통해 프롬프트를 불러옵니다.

In [18]:
config = {
    "_type": "few_shot",
    "input_variables": ["conversation_history"],
    "prefix_path": "../model/templates/user_response_prompt_prefix_template.txt",
    "example_prompt": {
        "_type": "prompt",
        "input_variables": ["conversation_history", "user_responses"],
        "template": "conversation_history: \n{conversation_history}\n{user_responses}"
    },
    "examples": "../model/templates/examples/user_response_prompt_examples.json",
    "suffix_path": "../model/templates/user_response_prompt_suffix_template.txt"
}

In [19]:
# 파일로 부터 프롬프트 불러오기
template_path="../model/templates/user_response_prompt.json"
user_response_prompt = load_prompt_from_config(config)

아래와 같이 format 메서드로 프롬프트를 확인할 수 있습니다.

In [22]:
example_conversation_history = """
User: 안녕하세요. <END_OF_TURN>
이우선: 무엇을 도와드릴까요? <END_OF_TURN>
User: 와인 추천해주세요. <END_OF_TURN>
이우선: 어떤 행사나 기념일을 위해 와인을 찾으시는지 알려주실 수 있으신가요? <END_OF_TURN>
"""

In [23]:
print(
    user_response_prompt.format(
        conversation_history=example_conversation_history,
    )
)

You are a user living in Seoul talking to a wine chatbot which can offer recommendations and Q&A.
Consider full context of the conversation and generate candidate responses, separated by "|", that the current user can say.
Set up a specific situation to generate three candidate answers. The specific situation must be unique.

Below is few examples of how to do this task.

conversation_history: 
User: Recommend me a wine please <END_OF_TURN>
Agent: We need to know your tastes so we can recommend a wine. Which type of wine do you prefer? Please choose from red, white, ros?, or sparkling. <END_OF_TURN>

user_response: I think 레드 와인 would be nice|I love 화이트 와인|I wonder what a 로제 와인 would be like.

conversation_history: 
User: Please recommend me a wine please <END_OF_TURN>
Agent: We need to know your tastes before we can recommend a wine. Which type of wine do you prefer? Red, white, ros?, or sparkling? <END_OF_TURN>
User: I'll go with ros? <END_OF_TURN>
Agent: Great, you like ros?! Can yo

In [24]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0.5)
user_response_chain = LLMChain(
    llm=llm,
    prompt=user_response_prompt, 
    verbose=True,
    output_key="user_responses"
)

Chain을 실행하기 위해서는 run 메서드를 실행한다.

In [25]:
user_responses = user_response_chain.run(
    {'conversation_history': example_conversation_history,}
)



> Entering new  chain...
Prompt after formatting:
You are a user living in Seoul talking to a wine chatbot which can offer recommendations and Q&A.
Consider full context of the conversation and generate candidate responses, separated by "|", that the current user can say.
Set up a specific situation to generate three candidate answers. The specific situation must be unique.

Below is few examples of how to do this task.

conversation_history: 
User: Recommend me a wine please <END_OF_TURN>
Agent: We need to know your tastes so we can recommend a wine. Which type of wine do you prefer? Please choose from red, white, ros?, or sparkling. <END_OF_TURN>

user_response: I think 레드 와인 would be nice|I love 화이트 와인|I wonder what a 로제 와인 would be like.

conversation_history: 
User: Please recommend me a wine please <END_OF_TURN>
Agent: We need to know your tastes before we can recommend a wine. Which type of wine do you prefer? Red, white, ros?, or sparkling? <END_OF_TURN>
User: I'll go with ro

In [26]:
user_responses

'생일 파티를 위한 와인이에요.|결혼 기념일을 맞이하여 특별한 와인을 찾고 있어요.|친구들과의 모임을 위한 와인이에요.'